In [12]:
import os
import torch
import torch.nn as nn
from torchvision import transforms, models
from torchvision.models import efficientnet_v2_s
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

In [13]:
class CTRDataset(Dataset):
    def __init__(self, folder_path, transform=None, ctr_min=0.0, ctr_max=0.27):
        self.folder_path = folder_path
        self.image_files = os.listdir(folder_path)
        self.transform = transform
        self.ctr_min = ctr_min
        self.ctr_max = ctr_max

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        filename = self.image_files[idx]
        ctr, _, _ = filename.split("_", 2)
        ctr = float(ctr) / 100.0  # first convert to 0-1 scale
        # Min-Max Normalization
        ctr = (ctr - self.ctr_min) / (self.ctr_max - self.ctr_min)
        img_path = os.path.join(self.folder_path, filename)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor([ctr], dtype=torch.float32)

In [14]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


dataset = CTRDataset("filtered_images", transform)
val_size = int(0.2 * len(dataset))
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [15]:

model = efficientnet_v2_s(weights="IMAGENET1K_V1")

# Užšaldome visus sluoksnius
for param in model.parameters():
    param.requires_grad = False

# Leisti mokytis tik nuo features.7 ir classifier
for name, param in model.named_parameters():
    if name.startswith("features.7") or name.startswith("classifier"):
        param.requires_grad = True

# Pakeičiame klasifikatorių į regresijos galvą
model.classifier = nn.Sequential(
    nn.Linear(model.classifier[1].in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, 1)
)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=1e-3)

# Training loop
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for imgs, targets in train_loader:
        imgs, targets = imgs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)  # shape: [B, 1]
        loss = criterion(outputs, targets)  # both shapes: [B, 1]
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    all_preds, all_targets = [], []
    with torch.no_grad():
        for imgs, targets in val_loader:
            imgs, targets = imgs.to(device), targets.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

            all_preds.extend(outputs.cpu().numpy().flatten())
            all_targets.extend(targets.cpu().numpy().flatten())

    # Metrics
    avg_train_loss = running_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    mae = mean_absolute_error(all_targets, all_preds)
    r2 = r2_score(all_targets, all_preds)

    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

Epoch 1, Train Loss: 0.0269, Val Loss: 0.0140, MAE: 0.0856, R²: -0.3232
Epoch 2, Train Loss: 0.0138, Val Loss: 0.0106, MAE: 0.0705, R²: -0.0087
Epoch 3, Train Loss: 0.0107, Val Loss: 0.0104, MAE: 0.0710, R²: 0.0139
Epoch 4, Train Loss: 0.0088, Val Loss: 0.0125, MAE: 0.0754, R²: -0.2016
Epoch 5, Train Loss: 0.0091, Val Loss: 0.0136, MAE: 0.0769, R²: -0.2710
Epoch 6, Train Loss: 0.0088, Val Loss: 0.0117, MAE: 0.0791, R²: -0.1156
Epoch 7, Train Loss: 0.0074, Val Loss: 0.0147, MAE: 0.0849, R²: -0.3777
Epoch 8, Train Loss: 0.0059, Val Loss: 0.0124, MAE: 0.0696, R²: -0.1868
Epoch 9, Train Loss: 0.0068, Val Loss: 0.0124, MAE: 0.0678, R²: -0.1804
Epoch 10, Train Loss: 0.0080, Val Loss: 0.0097, MAE: 0.0644, R²: 0.0980


In [ ]:
import os

# Define the path
path = r"C:\Users\benas\OneDrive\Desktop\University\Bachelor\solution\data\maxima\filtered_images"

# Supported image extensions
image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff", ".webp"}

# Count image files
image_count = sum(
    1 for file in os.listdir(path)
    if os.path.isfile(os.path.join(path, file)) and os.path.splitext(file)[1].lower() in image_extensions
)

print(f"Number of images: {image_count}")

Number of images: 675


In [10]:
with open("params.txt", "w") as f:
    for name, param in model.named_parameters():
        f.write(name + "\n")